In [1]:
from datetime import datetime
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
import matplotlib.pyplot as plt
from matplotlib import ticker, patheffects
from metpy.units import units
import numpy as np
from scipy.ndimage import gaussian_filter, maximum_filter, minimum_filter
import xarray as xr
import pandas as pd

In [4]:
import cdsapi

c = cdsapi.Client()

# Datas e horários
dias = ['20240429', '20240430', '20240501', '20240502', '20240503']
horas = ['00:00', '06:00', '12:00', '18:00']

c.retrieve(
    'reanalysis-era5-pressure-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'pressure_level': [
            '1000', '925', '850', '700', '500', '400', '300', '250', '200', '150', '100'
        ],
        'variable': [
            'geopotential', 'temperature',
            'u_component_of_wind', 'v_component_of_wind'
        ],
        'year': ['2024'],
        'month': ['04', '05'],
        'day': ['29', '30', '01', '02', '03'],
        'time': horas,
        'area': [15, -85, -60, -30],  # [Norte, Oeste, Sul, Leste] - América do Sul
    },
    'era5_geopotential_tendency_2024.nc'
)


2025-06-21 16:55:42,524 INFO [2025-06-16T00:00:00] CC-BY licence to replace Licence to use Copernicus Products on 02 July 2025. More information available [here](https://forum.ecmwf.int/t/cc-by-licence-to-replace-licence-to-use-copernicus-products-on-02-july-2025/13464)
2025-06-21 16:55:42,524 INFO [2025-06-10T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-06-21 16:55:42,525 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-06-21 16:55:43,374 INFO Request ID is 678f7163-a5ae-42f3-9cc8-b5eef0453d8b
2025-06-21 16:55:43,690 INFO status has been updated to accepted
2025-06-21 16:55:52,902 INFO status has been updated to running
2025-06-21 16:57:40,032 INFO status has been updated to successful


'era5_geopotential_tendency_2024.nc'

In [18]:
import numpy as np
from metpy.constants import Rd, g
from metpy.calc import potential_temperature
from metpy.units import units

def coriolis_parameter(lat):
    """Calcula o parâmetro de Coriolis f."""
    omega = 7.2921e-5  # rad/s
    return 2 * omega * np.sin(np.radians(lat))

def static_stability(T, p, theta=None):
    """Calcula a estabilidade estática σ = - (R * T / p) * d(lnθ)/dp"""
    if theta is None:
        theta = potential_temperature(p * units.Pa, T * units.K)
    dlnθ_dp = np.gradient(np.log(theta), p, axis=0)
    sigma = - (Rd * T / p) * dlnθ_dp
    return sigma

def termo_A_lhs(topo, p, sigma, f0, dx, dy):
    """Termo A: Operador do lado esquerdo da equação"""
    laplaciano = (
        np.gradient(np.gradient(topo, dx, axis=-1), dx, axis=-1) +
        np.gradient(np.gradient(topo, dy, axis=-2), dy, axis=-2)
    )
    d_sigma = np.gradient((f0**2 / sigma) * np.gradient(topo, p, axis=0), p, axis=0)
    return laplaciano + d_sigma

def termo_B(Vg, phi, f, f0, dx, dy):
    """Termo B: -f0 * Vg · ∇[(1/f0) ∇²φ + f]"""
    lap_phi = (
        np.gradient(np.gradient(phi, dx, axis=-1), dx, axis=-1) +
        np.gradient(np.gradient(phi, dy, axis=-2), dy, axis=-2)
    )
    escalar = (1 / f0) * lap_phi + f
    dphidx = np.gradient(escalar, dx, axis=-1)
    dphidy = np.gradient(escalar, dy, axis=-2)
    return -f0 * (Vg[0] * dphidx + Vg[1] * dphidy)

def termo_C(Vg, phi, sigma, f0, p, dx, dy):
    """Termo C: -∂/∂p [f0² / σ * Vg · ∇( -∂φ/∂p )]"""
    dphidp = -np.gradient(phi, p, axis=0)
    dtermx = np.gradient(dphidp, dx, axis=-1)
    dtermy = np.gradient(dphidp, dy, axis=-2)
    produto = (f0**2 / sigma) * (Vg[0] * dtermx + Vg[1] * dtermy)
    return -np.gradient(produto, p, axis=0)

def termo_D(T, sigma, f0, p):
    """Termo D: -f0² / σ * d/dp (κ / σ * dT/dp)"""
    kappa = 0.286  # R/cp para ar seco
    dTdp = np.gradient(T, p, axis=0)
    termo = (kappa / sigma) * dTdp
    dtermo_dp = np.gradient(termo, p, axis=0)
    return -f0**2 / sigma * dtermo_dp


In [ ]:
import xarray as xr

ds = xr.open_dataset("era5_geopotential_tendency_2024.nc")
T = ds['t'].values          # temperatura em K
z = ds['z'].values          # geopotencial em m^2/s^2
lat = ds['latitude'].values
lon = ds['longitude'].values
p = ds['pressure_level'].values * 100  # hPa → Pa

from metpy.units import units
from metpy.constants import g

# Adiciona unidades ao geopotencial
z = z * units('m^2 / s^2')

# Converte para altura geopotencial (em metros)
height = z / g  # agora height tem unidades de metros (m)


In [21]:
import numpy as np
from metpy.calc import lat_lon_grid_deltas
from metpy.units import units

# Grade 2D
lat_2d, lon_2d = np.meshgrid(lat, lon, indexing='ij')

# Coriolis
def coriolis_parameter(lat):
    omega = 7.2921e-5  # rad/s
    return 2 * omega * np.sin(np.radians(lat))

f = coriolis_parameter(lat_2d) * units('1/s')
f0 = coriolis_parameter(np.mean(lat))

# Deltas espaciais
dx, dy = lat_lon_grid_deltas(lon_2d, lat_2d, x_dim=-1, y_dim=-2)
dx_val = np.mean(dx)
dy_val = np.mean(dy)
